# ETL – Transformation RDF vers CSV (Chicago)

Objectif :
- Charger les données RDF de Chicago
- Extraire les informations de ridership
- Transformer les données en format tabulaire
- Exporter un CSV propre pour Power BI


In [1]:
import pandas as pd
import numpy as np
from rdflib import Graph


Chargement du fichier RDF

In [7]:
import os
from rdflib import Graph

# Dossier contenant les fichiers RDF
rdf_folder = "../data/raw/chicago/rdf/rdf_CTA__Ridership__Daily_by_Route_routes_2001_2025/"

# Création du graphe global
g = Graph()

file_count = 0

# Chargement de tous les fichiers RDF
for file in os.listdir(rdf_folder):
    if file.endswith(".rdf"):
        file_path = os.path.join(rdf_folder, file)
        g.parse(file_path)
        file_count += 1

print(f"Nombre de fichiers RDF chargés : {file_count}")
print(f"Nombre total de triplets RDF : {len(g)}")


Nombre de fichiers RDF chargés : 19
Nombre total de triplets RDF : 7647318


Explorer le contenu RDF

In [8]:
# Aperçu de quelques triplets RDF
for i, triple in enumerate(g):
    if i == 10:
        break
    print(triple)


(rdflib.term.URIRef('https://data.cityofchicago.org/resource/jyb9-n7fm/row-307896'), rdflib.term.URIRef('https://data.cityofchicago.org/resource/jyb9-n7fm/daytype'), rdflib.term.Literal('W'))
(rdflib.term.URIRef('https://data.cityofchicago.org/resource/jyb9-n7fm/row-532777'), rdflib.term.URIRef('https://data.cityofchicago.org/resource/jyb9-n7fm/rides'), rdflib.term.Literal('5503', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://data.cityofchicago.org/resource/jyb9-n7fm/row-411218'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://data.cityofchicago.org/resource/jyb9-n7fm'))
(rdflib.term.URIRef('https://data.cityofchicago.org/resource/jyb9-n7fm/row-817393'), rdflib.term.URIRef('http://www.socrata.com/rdf/terms#rowID'), rdflib.term.Literal('row-817393'))
(rdflib.term.URIRef('https://data.cityofchicago.org/resource/jyb9-n7fm/row-483074'), rdflib.term.URIRef('http://www.w3.org/1999/02/

Ajoute cette cellule (exploration ciblée)

In [9]:
from collections import Counter

predicates = [str(p) for _, p, _ in g]
Counter(predicates).most_common(15)


[('https://data.cityofchicago.org/resource/jyb9-n7fm/daytype', 1092474),
 ('https://data.cityofchicago.org/resource/jyb9-n7fm/rides', 1092474),
 ('http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 1092474),
 ('http://www.socrata.com/rdf/terms#rowID', 1092474),
 ('http://www.w3.org/2000/01/rdf-schema#member', 1092474),
 ('https://data.cityofchicago.org/resource/jyb9-n7fm/date', 1092474),
 ('https://data.cityofchicago.org/resource/jyb9-n7fm/route', 1092474)]

Extraction RDF → dictionnaire Python

In [10]:
records = []

for s, p, o in g:
    records.append({
        "subject": str(s),
        "predicate": str(p),
        "object": str(o)
    })

df_rdf = pd.DataFrame(records)
df_rdf.head()


,subject,predicate,object
0,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,W
1,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,5503
2,https://data.cityofchicago.org/resource/jyb9-n...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://data.cityofchicago.org/resource/jyb9-n7fm
3,https://data.cityofchicago.org/resource/jyb9-n...,http://www.socrata.com/rdf/terms#rowID,row-817393
4,https://data.cityofchicago.org/resource/jyb9-n...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://data.cityofchicago.org/resource/jyb9-n7fm


Filtrer les champs utiles

In [11]:
df_rdf[df_rdf["predicate"].str.contains("date|route|ride|board", case=False, regex=True)].head(20)


,subject,predicate,object
1,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,5503
8,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,2002-09-04T00:00:00
9,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,136
10,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,2021-06-12T00:00:00
23,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,X49
24,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,2015-03-14T00:00:00
25,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,22927
26,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,22
27,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,1205
28,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,2024-04-15T00:00:00


Conclusion Jusqu'a Ici

“J’ai exploré les prédicats RDF pour identifier les champs métier pertinents avant de les transformer en données tabulaires.”

Construire le DataFrame final (version simple)

In [12]:
df_clean = df_rdf[
    df_rdf["predicate"].str.contains("date|route|ride|board", case=False, regex=True)
]

df_clean.head()


,subject,predicate,object
1,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,5503
8,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,2002-09-04T00:00:00
9,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,136
10,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,2021-06-12T00:00:00
23,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,X49


## transformer le RDF en table exploitable (CSV).

Extraire le type d’info depuis predicate

In [13]:
# Identifier le type d'information
def extract_feature(predicate):
    if "date" in predicate:
        return "date"
    elif "route" in predicate:
        return "route"
    elif "board" in predicate:
        return "boardings"
    elif "ride" in predicate:
        return "rides"
    else:
        return None

df_clean["feature"] = df_clean["predicate"].apply(extract_feature)
df_clean.head()


C:\Users\user\AppData\Local\Temp\ipykernel_4776\2035203407.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["feature"] = df_clean["predicate"].apply(extract_feature)


,subject,predicate,object,feature
1,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,5503,rides
8,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,2002-09-04T00:00:00,date
9,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,136,route
10,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,2021-06-12T00:00:00,date
23,https://data.cityofchicago.org/resource/jyb9-n...,https://data.cityofchicago.org/resource/jyb9-n...,X49,route


Passer de format RDF → format table (pivot)

In [14]:
df_pivot = (
    df_clean
    .pivot_table(
        index="subject",
        columns="feature",
        values="object",
        aggfunc="first"
    )
    .reset_index()
)

df_pivot.head()


feature,subject,date,rides,route
0,https://data.cityofchicago.org/resource/jyb9-n...,2001-01-02T00:00:00,5813,1
1,https://data.cityofchicago.org/resource/jyb9-n...,2001-01-03T00:00:00,6809,1
2,https://data.cityofchicago.org/resource/jyb9-n...,2001-01-16T00:00:00,7775,1
3,https://data.cityofchicago.org/resource/jyb9-n...,2001-05-22T00:00:00,7429,1
4,https://data.cityofchicago.org/resource/jyb9-n...,2004-12-02T00:00:00,3925,1


Nettoyage & typage

In [17]:
df_pivot["date"] = pd.to_datetime(df_pivot["date"], errors="coerce")
df_pivot["rides"] = pd.to_numeric(df_pivot["rides"], errors="coerce")

df_pivot.info()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092474 entries, 0 to 1092473
Data columns (total 4 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   subject  1092474 non-null  object        
 1   date     1092474 non-null  datetime64[ns]
 2   rides    1092474 non-null  int64         
 3   route    1092474 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 33.3+ MB


In [19]:
# Vérification rapide des valeurs
df_pivot.head()

feature,subject,date,rides,route
0,https://data.cityofchicago.org/resource/jyb9-n...,2001-01-02,5813,1
1,https://data.cityofchicago.org/resource/jyb9-n...,2001-01-03,6809,1
2,https://data.cityofchicago.org/resource/jyb9-n...,2001-01-16,7775,1
3,https://data.cityofchicago.org/resource/jyb9-n...,2001-05-22,7429,1
4,https://data.cityofchicago.org/resource/jyb9-n...,2004-12-02,3925,1


In [20]:
# Nettoyage des données
df_pivot = df_pivot.dropna(subset=["date", "route", "rides"])


NOTEBOOK 01 — Export CSV

In [22]:
output_path = "../data/processed/cta_ridership_by_route_daily.csv"
df_pivot.to_csv(output_path, index=False)

print("CSV exporté :", output_path)


CSV exporté : ../data/processed/cta_ridership_by_route_daily.csv
